In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import dill
import random
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error, make_scorer,mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_predict
from xgboost import XGBRegressor

from patsy import dmatrices, dmatrix, build_design_matrices

# Set number of CPU cores for parallel algorithms
import os
if "CPU_LIMIT" in os.environ:
    # If you are on JupyterHub, this gives you the right number of CPUs for your virtual machine
    num_cpus = int(os.getenv("CPU_LIMIT").split('.')[0])
else:
    # If you are not on JupyterHub, this gives you the right number for your computer.
    num_cpus = os.cpu_count()

In [2]:
df = pd.read_csv("topics_pm_wo_facts_having_txns_inv_release_filtered.csv")

In [4]:
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.DataFrame({'Variable Name': df.columns,
                                   'Data Type': df.dtypes,
                                   'Missing Values': df.isnull().sum(),
                                   'Unique Values': [df[name].nunique() for name in df.columns]}
                                 ).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [5]:
summarize_dataframe(df)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0.1,int64,0,48367,48367.0,,,,24183.0,13962.494572,0.0,12091.5,24183.0,36274.5,48366.0
Unnamed: 0,int64,0,48367,48367.0,,,,24183.0,13962.494572,0.0,12091.5,24183.0,36274.5,48366.0
workorderid,object,0,48367,48367.0,48367,1462943,1,,,,,,,
wostatus,object,0,6,48367.0,6,CLOSED,47239,,,,,,,
wodesc,object,0,13596,48367.0,13596,VEHICLE STATE INSPECTION (FACILITIES MANAGEMEN...,625,,,,,,,
woproject,object,25294,32,23073.0,32,1034.0,8892,,,,,,,
wotypedesc,object,0,2,48367.0,2,Maintenance services including utility systems...,48363,,,,,,,
wodatecreated,object,0,1258,48367.0,1258,8/16/21,478,,,,,,,
wocategorywithdesc,object,0,5,48367.0,5,PM CODE (Code related Preventive maintenance w...,26869,,,,,,,
wopropertyid,object,0,672,48367.0,672,VHCL,2670,,,,,,,


In [6]:
df["Name_spent"]= df[["Name","amount_spent_material"]].groupby('Name')['amount_spent_material'].transform('sum').drop_duplicates()

with pd.option_context('display.max_rows', None):
    print(df[["Name","Name_spent"]].drop_duplicates().sort_values(by='Name_spent').reset_index(drop=True) )


                                           Name  Name_spent
0                           25_walk_res_rm_sand     1724.61
1               28_clock_reset_ptac_maintenance     1814.43
2                   24_glove_voltage_rated_test     6192.78
3                 23_outlets_fittings_gas_power     8012.86
4   26_collection_systems_institute_collections    22563.75
5                  27_upload_jci_archive_device    35203.39
6                          20_boxes_vav_vfd_aed    37100.70
7                   17_split_system_glycol_semi    48811.39
8                  21_panelboard_years_year_dim    60670.90
9                    22_wax_june_floor_december    96537.26
10          12_ice_machine_refrigerator_freezer   115143.89
11           18_grease_traps_turnover_lubricate   126468.10
12                       15_tank_gas_med_polish   140342.76
13           19_termite_renewal_dodson_warranty   147020.15
14           16_cleaning_window_sweeping_street   203742.49
15             13_filter_replacement_wat

In [7]:
df['date_column'] = pd.to_datetime(df['wodatecreated'])
df['max_labor'] = pd.to_datetime(df['max_labor_complete_date'])


# Extract month and year into separate columns
df['year_wo_created'] = df['date_column'].dt.year
df['month_wo_created'] = df['date_column'].dt.month
df['year_wo_labor_complete'] = df['max_labor'].dt.year
df['month_wo_labor_complete'] = df['max_labor'].dt.month

df['years_to_labor_complete'] = df['year_wo_labor_complete'] - df['year_wo_created']
df['ct_trxn_non_inv'] = df['cnt_txns'] - df['cnt_inv_releases']
df['pct_inv_release'] = df['cnt_inv_releases'] / df['cnt_txns']
df = df[df["Topic"].notnull()]
df["cnt_pos"] = df["cnt_pos"].fillna(0)

/tmp/ipykernel_79/1336917662.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_column'] = pd.to_datetime(df['wodatecreated'])


In [7]:
lm_1 = smf.ols(formula='amount_spent_material ~ years_to_labor_complete + wopropertydescription + cnt_pos + cnt_inv_releases + Name + amount_spent_labor + year_wo_created', data=df).fit()
lm_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     amount_spent_material   R-squared:                       0.271
Model:                               OLS   Adj. R-squared:                  0.261
Method:                    Least Squares   F-statistic:                     26.95
Date:                   Tue, 23 Jul 2024   Prob (F-statistic):               0.00
Time:                           02:37:58   Log-Likelihood:            -3.6645e+05
No. Observations:                  47892   AIC:                         7.342e+05
Df Residuals:                      47240   BIC:                         7.399e+05
Df Model:                            651                                         
Covariance Type:               nonrobust                                         
===========================================================================================================================================================
                                                                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                               -2.411e+04   1799.317    -13.397      0.000   -2.76e+04   -2.06e+04
wopropertydescription[T.1003 WEST MAIN]                                                    21.3205    367.202      0.058      0.954    -698.401     741.042
wopropertydescription[T.101 ELLIEWOOD AVE]                                               -140.5590    512.751     -0.274      0.784   -1145.558     864.440
wopropertydescription[T.1018 WEST MAIN ST]                                                -34.7209    400.974     -0.087      0.931    -820.635     751.193
wopropertydescription[T.102 CRESAP ROAD]                                                  -69.0996    411.306     -0.168      0.867    -875.266     737.066
wopropertydescription[T.108 CRESAP ROAD]                                                 -229.9551    394.278     -0.583      0.560   -1002.746     542.836
wopropertydescription[T.11TH STREET GENERATOR FARM]                                      3962.1616    627.967      6.310      0.000    2731.337    5192.986
wopropertydescription[T.11TH STREET PARKING GARAGE]                                       203.4199    365.858      0.556      0.578    -513.668     920.508
wopropertydescription[T.1200 West Main St.]                                               -31.1819    627.908     -0.050      0.960   -1261.891    1199.528
wopropertydescription[T.1202 WEST MAIN ST]                                                -31.1819    627.908     -0.050      0.960   -1261.891    1199.528
wopropertydescription[T.1413 1/2 UNIVERSITY AVE (Chancellor Apartments)]                 -176.5543    628.217     -0.281      0.779   -1407.868    1054.759
wopropertydescription[T.1515 UNIVERSITY AVE]                                              -13.6203    367.231     -0.037      0.970    -733.399     706.158
wopropertydescription[T.199 SPOTNAP ROAD]                                                -157.0363    512.876     -0.306      0.759   -1162.281     848.209
wopropertydescription[T.2014 GOOSE CREEK ROAD (UVA DERMATOLOGY WAYNESBORO)]              -162.6576    468.128     -0.347      0.728   -1080.196     754.880
wopropertydescription[T.2015 IVY ROAD (DYNAMICS BLDG)]                                     -1.2611    382.283     -0.003      0.997    -750.541     748.019
wopropertydescription[T.2033 IVY ROAD]                                                    -43.2476    627.910     -0.069      0.945   -1273.960    1187.465
wopropertydescription[T.2211 HYDRAULIC ROAD]                                          

In [8]:
lm_1 = smf.ols(formula='amount_spent_material ~ years_to_labor_complete + cnt_pos + cnt_inv_releases + amount_spent_labor + year_wo_created', data=df).fit()
lm_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     amount_spent_material   R-squared:                       0.156
Model:                               OLS   Adj. R-squared:                  0.156
Method:                    Least Squares   F-statistic:                     1767.
Date:                   Tue, 23 Jul 2024   Prob (F-statistic):               0.00
Time:                           23:05:20   Log-Likelihood:            -3.6995e+05
No. Observations:                  47892   AIC:                         7.399e+05
Df Residuals:                      47886   BIC:                         7.400e+05
Df Model:                              5                                         
Covariance Type:               nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept               -2.477e+04   1775.992    -13.945      0.000   -2.82e+04   -2.13e+04
years_to_labor_complete   144.0457      4.880     29.517      0.000     134.481     153.611
cnt_pos                   106.7480      1.286     82.995      0.000     104.227     109.269
cnt_inv_releases           -5.3415      1.922     -2.779      0.005      -9.109      -1.573
amount_spent_labor          0.0101      0.001      8.662      0.000       0.008       0.012
year_wo_created            12.3605      0.880     14.051      0.000      10.636      14.085
==============================================================================
Omnibus:                    39290.737   Durbin-Watson:                   1.745
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1289247.363
Skew:                           3.795   Prob(JB):                         0.00
Kurtosis:                      27.258   Cond. No.                     1.74e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
lm_3 = smf.ols(formula='years_to_labor_complete ~ cnt_pos + cnt_inv_releases + amount_spent_labor + year_wo_created', data=df).fit()
lm_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     amount_spent_material   R-squared:                       0.156
Model:                               OLS   Adj. R-squared:                  0.156
Method:                    Least Squares   F-statistic:                     1767.
Date:                   Tue, 23 Jul 2024   Prob (F-statistic):               0.00
Time:                           23:34:01   Log-Likelihood:            -3.6995e+05
No. Observations:                  47892   AIC:                         7.399e+05
Df Residuals:                      47886   BIC:                         7.400e+05
Df Model:                              5                                         
Covariance Type:               nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept               -2.477e+04   1775.992    -13.945      0.000   -2.82e+04   -2.13e+04
years_to_labor_complete   144.0457      4.880     29.517      0.000     134.481     153.611
cnt_pos                   106.7480      1.286     82.995      0.000     104.227     109.269
cnt_inv_releases           -5.3415      1.922     -2.779      0.005      -9.109      -1.573
amount_spent_labor          0.0101      0.001      8.662      0.000       0.008       0.012
year_wo_created            12.3605      0.880     14.051      0.000      10.636      14.085
==============================================================================
Omnibus:                    39290.737   Durbin-Watson:                   1.745
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1289247.363
Skew:                           3.795   Prob(JB):                         0.00
Kurtosis:                      27.258   Cond. No.                     1.74e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""